# Projekat: Ball and Beam

## Potrebne biblioteke

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

## Okruženje

In [ ]:
class Enviroment():
    # m - masa loptice (0.1113 kg je preporucena vrednost)
    # r - poluprecnik loptice (0.015 m je preporucena vrednost)
    # l - duzina grede (0.4 m je preporucena vrednost)
    # TIME_STEP - za diskretizaciju vremena (0.05 s je preporucena vrednost)
    # J - moment inercije za lopticu (J = 2/5 * m * r ^ 2)
    # alpha - ugao rotacije (ugao u radijanima)
    # g - gravitaciona konstanta (9.81 m/s^2 je preporucena vrednost)
    # kf - koeficijent trenja (0.5 je preporucena vrednost uz pretpostavku da je metalna loptica i drvena greda)
    # p - koeficijent kaznjavanja (1 je preporucena vrednost)
    def __init__(self, m: float = 0.113, r: float = 0.015, l: float = 0.4, g: float = 9.81, kf: float = 0.5, TIME_STEP: float = 0.05):
        self.m = m
        self.r = r
        self.l = l
        self.J = 0.4 * m * r ** 2
        self.g = g
        self.kf = kf
        self.TIME_STEP = TIME_STEP
        self.p = 1

        # Loptica se nalazi na sredini grede na pocetku i greda je u ravnoteznom polozaju
        # Ovo je u sustini stanje z = (z1, z2, z3, z4)
        self.x = 0
        self.x_dot = 0
        self.alpha = 0
        self.alpha_dot = 0

        # Ogranicenja maksimalnih vrednosti
        self.x_dot_max = 0.5
        self.alpha_max = 0.35
        self.alpha_dot_max = 1
        
        # Ako predjemo 500 koraka vraca se true za truncated (ogranicavamo duzinu epizode)
        self.steps = 0
        self.max_steps = 500
        self.trunc = False
        # Da li je loptica pala?
        self.term = False

        # Ugaono ubrzanje predstavlja akciju (rad/s^2)
        # Skup akcija
        self.actions = [-5, -2.5, 0, 2.5, 5]

    # Naredna funkcija je formula sa pdf-a
    def x_ddot(self):
        return - self.x_dot * (self.kf * self.r ** 2) / (self.m * self.r ** 2 + self.J) + np.sin(self.alpha) * (self.m * self.g * self.r ** 2) / (self.m * self.r ** 2 + self.J)

    # Funkcija nagrade dobijena primenom DRM bazirana na evaluacionim funkcijama
    def reward(self):
        reward = self.e_0() * self.e_1() * self.e_2()
        return reward
    
    # Evaluacione funkcije, date su u pdf-u
    def e_0(self):
        if self.x > self.l / 2 or self.x < - self.l / 2:
            return 0
        if self.x_dot > self.x_dot_max or self.x_dot < - self.x_dot_max:
            return 0
        if self.alpha > self.alpha_max or self.alpha < - self.alpha_max:
            return 0
        if self.alpha_dot > self.alpha_dot_max or self.alpha_dot < - self.alpha_dot_max:
            return 0
        return 1

    def e_1(self):
        return 1 - 2 * abs(self.x) / self.l

    def e_2(self):
        return 1 - self.p * abs(self.x_ddot()) 

    # alpha_ddot - akcija (ugaono ubrzanje) koje primenjujemo
    # action - akcija koju uzimamo
    # action predstavlja indeks unutar actions tako da indeksi [0,1,2,3,4] odgovaraju akcijama [-5, -2.5, 0, 2.5, 5]
    def take_action(self, action: int):
        self.steps += 1
        if self.steps > self.max_steps:
            self.trunc = True   # Znaci nije terminalno stanje (pad sipke), vec smo prosto dosli do kraja epizode
            self.term = False
            return [self.x, self.x_dot, self.alpha, self.alpha_dot], self.reward(), self.trunc, self.term

        # U slucaju da je loptica pala sa grede
        if self.x > self.l / 2 or self.x < - self.l / 2:
            self.term = True
            return [self.x, self.x_dot, self.alpha, self.alpha_dot], self.reward(), self.trunc, self.term
        
        # Primenom akcije menja se stanje
        alpha_ddot = self.actions[action]
        # Stanje je z = (z1, z2, z3, z4) pa je promena stanja definisana u naredne 4 linije
        self.x = self.x + self.TIME_STEP * self.x_dot
        self.x_dot = self.x_dot + self.TIME_STEP * self.x_ddot()
        self.alpha = self.alpha + self.TIME_STEP * self.alpha_dot
        self.alpha_dot = self.alpha_dot + self.TIME_STEP * alpha_ddot

        return [self.x, self.x_dot, self.alpha, self.alpha_dot], self.reward(), self.trunc, self.term

    #Reset okruzenja kako bi se mogla odigrati nova epizoda, resetuje se stanje, kako se epizoda zavrsila i korak u epizodi 
    def reset(self):
        self.x = 0
        self.x_dot = 0
        self.alpha = 0
        self.alpha_dot = 0
        self.steps = 0
        self.trunc = False
        self.term = False
        return [self.x, self.x_dot, self.alpha, self.alpha_dot], self.reward(), self.trunc, self.term

In [3]:
env = Enviroment(m = 0.113, r = 0.015, l = 0.4, g = 9.81, kf = 0.5, TIME_STEP = 0.05)

## Diskretizacija

In [4]:
# Broj binova za svaku dimenziju stanja
N_X_BINS = 20
N_X_DOT_BINS = 20
N_ALPHA_BINS = 20
N_ALPHA_DOT_BINS = 20

# Granice za diskretizaciju
# Stavljao sam isto ogranicenje kao da su thresholdi, koji su definisani u e0 u pdf fajlu :)

# Ove promenljive su dodate kako bi se lakse menjale njihove vrednosti ako treba :)
x_th = 0.2
x_dot_th = 0.5
alpha_th = 0.35
alpha_dot_th = 1

# Granice za poziciju
x_bins = np.linspace(-x_th, x_th, N_X_BINS + 1)
# Granice brzina - namesteno da se sistem ponasa "realno"
x_dot_bins = np.linspace(-x_dot_th, x_dot_th, N_X_DOT_BINS + 1)
# Granice za ugao u radijanima
alpha_bins = np.linspace(-alpha_th, alpha_th, N_ALPHA_BINS + 1)  
# Granice za ugaonu brzinu moraju biti male kako bi sistem bio stabilan
alpha_dot_bins = np.linspace(-alpha_dot_th, alpha_dot_th, N_ALPHA_DOT_BINS + 1)

# Funkcija za konverziju kontinualnog stanja u diskretni indeks
def discretize_state(state):
    x, x_dot, alpha, alpha_dot = state
    
    # clip sprecava pojavu ekstremnih vrednosti tako sto neki autsajder postave na njemu najblizu granicu
    x = np.clip(x, -x_th, x_th)
    x_dot = np.clip(x_dot, -x_dot_th, x_dot_th)
    alpha = np.clip(alpha, -alpha_th, alpha_th)
    alpha_dot = np.clip(alpha_dot, -alpha_dot_th, alpha_dot_th)
    
    # digitize - vraca vrednost bina u koji spada vrednost
    x_idx = np.digitize(x, x_bins) - 1
    x_dot_idx = np.digitize(x_dot, x_dot_bins) - 1
    theta_idx = np.digitize(alpha, alpha_bins) - 1
    theta_dot_idx = np.digitize(alpha_dot, alpha_dot_bins) - 1
    
    # clamp - osigurava da indeks nikad ne izadje van validnog opsega
    x_idx = max(0, min(x_idx, N_X_BINS - 1))
    x_dot_idx = max(0, min(x_dot_idx, N_X_DOT_BINS - 1))
    alpha_idx = max(0, min(theta_idx, N_ALPHA_BINS - 1))
    alpha_dot_idx = max(0, min(theta_dot_idx, N_ALPHA_DOT_BINS - 1))
    
    return (x_idx, x_dot_idx, alpha_idx, alpha_dot_idx)

## Treniranje agenata

### Vizualizacija treniranja

In [5]:
# Zamislio sam da ovde definišeš funkciju koja će ti kreirati one silne grafike kao kod cart_pole :)

### Q-Learning agent

In [6]:
# U ovoj sekciji napraviš Q-Learning agenta

### SARSA agent

In [7]:
# U ovoj sekciji napraviš SARSA agenta

## Zaključak

MOŽDA. Ovde ćemo samo zapisati zaključke koje smo izveli iz rada nad ovim zadatkom. :)